In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import decomposition
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('train/features.csv',header = None)
df.head(6)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,-424.004655,85.886018,-19.391035,19.548873,-3.152061,-3.135796,-5.586479,2.204532,-0.968167,-4.146447,...,0.756196,0.754540,18.187436,14.309552,16.744301,16.287958,16.495040,18.176403,53.666413,down
1,-427.845940,61.444910,-69.906503,75.739853,-35.569725,9.582659,6.669224,5.555409,4.426516,9.670583,...,0.789944,0.717734,23.542518,16.525604,19.920645,17.190587,18.389313,21.918735,60.188674,down
2,-327.288344,107.655145,-85.208783,-4.627603,-34.472271,-2.624659,-18.956204,-6.853750,-7.975197,-20.111568,...,0.553500,0.663733,31.389790,14.436285,17.012344,18.676632,18.853558,19.082480,57.331269,down
3,-341.501762,107.211360,-90.836330,-0.098215,-33.703336,-3.610792,-29.906583,-9.599740,-18.541331,-23.105156,...,0.536866,0.596243,29.135632,14.979695,17.966184,17.527041,17.873409,20.326471,57.930192,down
4,-442.097720,142.172651,-25.851881,33.580954,-4.230062,12.438396,-16.501494,12.557016,0.892619,-2.308974,...,0.361471,0.472670,26.285881,19.839796,20.518118,19.795706,18.634939,17.751166,56.695714,down
5,-481.401211,96.320072,-3.340789,6.039630,-1.719361,16.746752,-0.383682,-8.960226,6.744356,-1.369229,...,0.626463,0.570527,20.251495,15.690428,17.070937,16.076208,22.293984,21.240883,51.277368,down


In [4]:
df = shuffle(df)
df.head(6)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
28964,-335.978262,119.343931,-31.556878,21.588382,11.943723,-11.162789,-9.650644,1.143235,-4.690378,7.996593,...,0.608567,0.520504,24.211603,13.898853,16.801402,17.380736,17.666972,21.345059,56.609929,unknown
65076,-174.229126,70.875239,-25.236081,59.074328,-37.043027,45.512966,-2.877650,7.359790,5.802757,-6.613318,...,0.410268,0.475833,20.642822,17.670393,18.750376,19.728062,17.206548,20.664537,64.272461,yes
12889,-485.948707,89.861420,-9.683977,1.714240,-7.225182,-10.244950,19.978331,5.204700,-17.604245,-7.971003,...,0.659339,0.528439,26.933197,17.635034,20.698076,22.727343,24.816643,25.304431,45.423541,on
44885,-406.098347,94.245505,-1.417869,-3.838691,-20.689914,-3.661901,-19.350792,-7.318286,1.397389,-11.682958,...,0.607558,0.574360,26.443480,13.947896,15.464534,17.793195,19.455857,20.237050,50.422494,unknown
5525,-295.928910,133.384853,-4.339349,17.232813,4.892446,12.160908,1.568456,3.156511,-3.599015,2.155153,...,0.679018,0.687895,16.607177,12.379760,17.162683,16.610449,18.085300,18.563312,52.312026,left
22828,-375.447650,122.186609,-19.022811,-0.130575,-36.094570,-7.807593,-29.093190,-10.875411,-12.026430,-9.851988,...,0.593021,0.552610,25.052743,12.145103,14.914025,14.619561,16.663559,18.221305,56.785013,unknown


# Partitioning Data

In [6]:
tr = 60000

features = np.array(df)
tr_x = features[:tr,0:58]
tr_y = features[:tr,59]
val_x = features[tr:,0:58]
val_y = features[tr:,59]

# Preprocessing

In [7]:
tr_mean = np.mean(tr_x, axis=0)
tr_std = np.std(tr_x, axis = 0,dtype=np.float32)
tr_xp = (tr_x - tr_mean[None,:])/tr_std[None,:]

val_mean = np.mean(val_x, axis=0)
val_std = np.std(val_x, axis = 0,dtype=np.float32)
val_xp = (val_x - val_mean[None,:])/val_std[None,:]

lab = LabelEncoder()
tr_yp= lab.fit_transform(tr_y)
val_yp= lab.fit_transform(val_y)

n_words= np.unique(tr_yp)
n_classes = len(n_words)
n_features = tr_xp.shape[1]

label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(n_classes))
tr_yoh = label_binarizer.transform(tr_yp)
val_yoh = label_binarizer.transform(val_yp)

# Supervised Learning

# AdaBoost Ensemble

In [14]:
ada_class = AdaBoostClassifier()
ada_class.fit(tr_xp, tr_yp)
ada_class.score(val_xp, val_yp)

0.63810267421432754

# Random Forest

In [9]:
rf_class = RandomForestClassifier()
rf_class.fit(tr_xp, tr_yp)
rf_class.score(val_xp, val_yp)

0.64786258051922696

# K-NN

In [10]:
knn_class = KNeighborsClassifier()
rf_class.fit(tr_xp, tr_yp)
rf_class.score(val_xp, val_yp)

0.64961936365410888

# Principal Component Analysis

In [11]:
pca = decomposition.PCA(n_components= 'mle', svd_solver ='full')
pca.fit(tr_xp)
tr_pca = pca.transform(tr_xp)
val_pca = pca.transform(val_xp)

In [12]:
knn_class = KNeighborsClassifier()
knn_class.fit(tr_pca, tr_yp)
knn_class.score(val_pca, val_yp)

0.62697638102674214

In [13]:
rf_class = RandomForestClassifier()
rf_class.fit(tr_pca, tr_yp)
rf_class.score(val_pca, val_yp)

0.63517470232285766